In [29]:
import sys
import pickle
import json
import pandas as pd
import numpy as np
sys.path.append('../')
from sklearn.pipeline import Pipeline
from modules.transformers.CenterPosition import CenterPositionTransform
from modules.transformers.TextType import isdateTransform, isfloatTransform,istextTransform
from modules.transformers.textalignment import AlignmentTransform
pd.set_option('display.max_rows', 500)
# from trcode.line import Line
# from trcode import helper
# from trcode.info_word import InfoWord

In [30]:
SYMBOL = {
    "$": 0,
}
sfea_words = SYMBOL

def clean_word(word):
    """
    Clean up a word
    :param word: input wordText
    :return: cleaned up word string
    """
    cword = word.strip(' /)*%-_:;\'\"')
    cword = cword.replace(',', '')
    cword = cword.replace('.', '')
    cword = cword.replace('(', '')
    cword = cword.replace('$', '')
    return cword
def gen_raw_feature(words):
    """
    Input a word list from one page, output preliminary features for this page.
    :param words: A list of strings which are words from this page.
    :return: A list of preliminary features contains word frequencies and other features like word count.
                Need some cleanup afterwards.
    """
    word_count = 0
    number_count = 0
    fea_sym = [0]*(max(sfea_words.values())+1)
    for w in words:
        # Count '$'
        dn = w.count('$')
        fea_sym[sfea_words['$']] += dn
        if dn > 1:
            wl = w.split('$')
            w = wl[0]
            words += wl[1:]
        cw = clean_word(w)
        if not cw:
            continue
        # Increase word count
        word_count += 1
        # Check if numbers and increase number count
        if cw.isnumeric():
            number_count += 1
            continue
    # Construct other features
    if not word_count:
        fea_oth = [0, 0, 0]
    else:
        fea_oth = [word_count, number_count]
    return fea_sym + fea_oth

In [31]:
def read_pages(ticker):
    with open('C:\\Users\\juang\\Documents\\Stocks\\PDFs\\'+ticker+'\\all.json', 'r') as f:
        datastore = json.load(f)
    df = pd.DataFrame()
    for i in range(0, len(datastore['Pages'])):
        df = df.append(pd.DataFrame(datastore['Pages'][i]))
    df = df[df.index == 0].set_index('PageNumber')
    df.index = df.index.astype(int)
    df = df.sort_index()
    df['Dollar_Count'] = ''
    df['Word_Count'] = ''
    df['Number_Count'] = ''
    for i in range(0, len(df)):
        features = gen_raw_feature(df.WordText[i])
        df['Dollar_Count'][i] = features[0]
        df['Word_Count'][i] = features[1]
        df['Number_Count'][i] = features[2]
    return df

In [32]:
def read_words(ticker):
    with open('C:\\Users\\juang\\Documents\\Stocks\\PDFs\\'+ticker+'\\all.json', 'r') as f:
        datastore = json.load(f)
    df = pd.DataFrame()
    for i in range(0, len(datastore['Pages'])):
        df = df.append(pd.DataFrame(datastore['Pages'][i]))
    df = df[df.index != 0].set_index('PageNumber')
    df.index = df.index.astype(int)
    df = df.sort_index()
    return df

In [33]:
def predict_pages(df):
    array = [df.Number_Count/df.Word_Count, df.Dollar_Count]
    g = number_clf.predict_proba(np.transpose(array))
    m = text_clf.predict_proba(df.WordText)
    all_probas = np.hstack((m, g))
    all_probas
    return final_clf.predict(all_probas)

In [34]:
with open('text_clf.pickle', 'rb') as f:
    text_clf = pickle.load(f)
with open('number_clf.pickle', 'rb') as f:
    number_clf = pickle.load(f)
with open('final_clf.pickle', 'rb') as f:
    final_clf = pickle.load(f)

In [35]:
chk = read_pages('CHK')

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [36]:
predict_pages(chk)

array([0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0], dtype=int64)

In [37]:
chk_words = read_words('CHK').reset_index()
chk_words.head(1)

,PageNumber,WordText,X_Max,X_Min,Y_Max,Y_Min
0,0,10-Q,196.0,142.0,155.0,123.0


In [38]:
chk_words = read_words('CHK').reset_index()
chk_words = chk_words.reset_index()
chk_words['PageNumber'] = chk_words['PageNumber'].astype(int)
chk_words['index'] = chk_words['index'].astype(int)

In [39]:
intakePipeSteps = []
intakePipeSteps.append(('line',AlignmentTransform(['Y_Max','Y_Min','index','PageNumber'],output_name='line',sort_columns=['index','PageNumber','Y_Min'],sort_values=[True,True,True])))
intakePipeSteps.append(('CenterTransform',CenterPositionTransform()))
intakePipeSteps.append(('isfloat',isfloatTransform('WordText')))
intakePipeSteps.append(('isdate',isdateTransform('WordText')))
intakePipeSteps.append(('istext',istextTransform('WordText')))
combined = Pipeline(intakePipeSteps)
combined

Pipeline(steps=[('line', <modules.transformers.textalignment.AlignmentTransform object at 0x0000017D60467080>), ('CenterTransform', <modules.transformers.CenterPosition.CenterPositionTransform object at 0x0000017D60467748>), ('isfloat', <modules.transformers.TextType.isfloatTransform object at 0x0000017D604677B8>), ('isdate', <modules.transformers.TextType.isdateTransform object at 0x0000017D60467668>), ('istext', <modules.transformers.TextType.istextTransform object at 0x0000017D60467D68>)])

In [40]:
print(combined.named_steps['line'].transform(chk_words))

30362 8 TextAlignment in
       line
0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        0
30        0
31        0
32        0
33        0
34        0
35        0
36        0
37        0
38        0
39        0
40        0
41        0
42        0
43        0
44        0
45        0
46        0
47        0
48        0
49        0
50        0
51        0
52        0
53        0
54        0
55        0
56        0
57        0
58        0
59        0
60        0
61        0
62        0
63        0
64        0
65        0
66        0
67        0
68        0
69        0
70        0
71        0
72        0
73        0
74        0
75        0
76        0
77        0
78        0
79        0
80 